In [1]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

In [2]:
driver_path = './chromedriver.exe'

In [3]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [4]:
def biz_khan(query,driver_path):
    domain = '세계일보'
    driver = webdriver.Chrome(executable_path=driver_path)
    url_domain = 'https://search.naver.com/search.naver?where=news&query='
    news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1022&nso=&is_sug_officeid=0'
    url = url_domain + query + news_domain
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    # 기사 url을 list형태로 반환
    try:
        news_list = soup.select('#main_pack > section > div > div.group_news > ul')
        url_list = []
        news_list = news_list[0].find_all('li')
        for url in news_list:
            url_list.append(str(url.find_all('a')[0].get('data-url')))
    except:
        print('기사가 없습니다.')
        driver.close()
        return 0
    driver.close()
    
    head_list = []
    data_list = []
    
    for url in url_list:
        driver = webdriver.Chrome(executable_path=driver_path)  # for Windows
        try:
            driver.get(url)
            time.sleep(1)
        except:
            print('link error')
            continue
        # try문 안에서 각자 언론사에 맞게 수정
        try:
            soup = BeautifulSoup(driver.page_source,'html.parser')
            head_list.append(soup.select('#title_sns')[0].text)
            data = soup.find('article',class_='viewBox').text
            data_list.append(' '.join(data))
        except:
            print(url)
            print('None_type')
        driver.close()
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

biz = biz_khan('삼성전자 슬로건',driver_path)
biz

link error
http://fn.segye.com/articles/article.asp?aid=20120501001543&cid=0501010000000
None_type


,언론사,제목,내용
0,세계일보,"삼성전자 ""애니콜 브랜드 슬로건 교체""",\n 삼 성 전 자 가 6 년 만...
1,세계일보,삼성전자 베트남서 QLED TV 출시,\n 삼 성 전 자 가 베 트 남 에...
2,세계일보,"삼성전자, '2015 사랑의 김장 나누기' 행사 개최",\n \n \n \n \n \n \n \n 지 난 1 4 일 삼 성 전 자 ...
3,세계일보,[이코노 브리핑] LG전자 ‘한마음 동행 페스타’ 진행 외,\n L G 전 자 ‘ 한 마 음 동 행 페 스 타 ’ 진 행 \n ...
4,세계일보,"[웰컴 투 U월드]삼성전자의 부풀리기 오보, 경쟁사 의식한 꼼수(?)",\n 해 외 여 행 을 떠 나 는 ...
5,세계일보,"삼성전자서비스, ‘2020 KSQI’ 가전 A/S 부문 10년 연속 1위… 국내 최...","\n ‘ 플 러 스 원 케 어 ’ , ' H R M 원 격 진 단 ..."
6,세계일보,뉴노멀·뉴가전… 삼성·LG ‘집콕 시대’ 신제품 공개,"\n 삼 성 , I F A 불 참 소 비 자 와 ‘ 온 택 트 만..."
7,세계일보,드디어 개막한 IFA 2015…새로운 트렌드는?,"\n 삼 성 전 자 , 인 간 중 심 의 I o T 기 술 선 보..."
8,세계일보,[설왕설래] 특허전쟁,\n “ 기 업 과 국 가 의 가 장 중 요 한 생 존 전 략 은 ...


In [6]:
biz.to_csv('세계일보_크롤링_데이터.csv',encoding='utf-8-sig')